<a href="https://colab.research.google.com/github/AlekhSaxena/Perceptron/blob/main/Stable_Diffusion_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!pip install diffusers==0.11.1
!pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers

In [ ]:
!pip install --quiet ipython-autotime
%load_ext autotime

In [ ]:
## Pipeline for Image Generation

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)

In [ ]:
pipe = pipe.to('cuda')

In [ ]:
pipe.enable_attention_slicing()
pipe.enable_xformers_memory_efficient_attention()

In [ ]:
prompt = 'an apple'

In [ ]:
img = pipe(prompt).images[0]

In [ ]:
type(img)

In [ ]:
img

In [ ]:
prompt = 'photograph of an apple'
img = pipe(prompt).images[0]
img

In [ ]:
## Def Generating Multiple Images

In [ ]:
from PIL import Image

def grid_img(imgs, rows=1, cols=3, scale=1):
  assert len(imgs) == rows * cols

  w, h = imgs[0].size
  w, h = int(w * scale), int(h * scale)

  grid = Image.new('RGB', size = (cols * w, rows * h))
  grid_w, grid_h = grid.size

  for i, img in enumerate(imgs):
    img = img.resize((w, h), Image.ANTIALIAS)
    grid.paste(img, box=(i % cols * w, i // cols * h))
  return grid

In [ ]:
num_imgs = 3
prompt = 'photograph of an old car'
imgs = pipe(prompt, num_images_per_prompt=num_imgs).images
grid = grid_img(imgs, rows=1, cols=3, scale=0.75)
grid

In [ ]:
seed = 777
prompt = "photograph of a mountain landscape during sunset, stars in the sky"
generator = torch.Generator("cuda").manual_seed(seed)
h, w = 512, 768
img = pipe(prompt, height=h, width=w,guidance_scale=10, num_inference_steps=20,generator=generator).images[0]
img

In [ ]:
## Remove Black and white Photos
num_images = 3

prompt = 'photograph of an old car'
neg_prompt = 'bw photo'

imgs = pipe(prompt, negative_prompt = neg_prompt, num_images_per_prompt=num_images).images

grid = grid_img(imgs, rows = 1, cols = 3, scale=0.75)
grid

## Other Models Versions
earlier 1.4 CompVis/stable-diffusion-v1-4<br>
SD v1.5 runwayml/stable-diffusion-v1-5<br>
SD v2.1 stabilityai/stable-diffusion-2-1 <br>



SD 1.5

In [ ]:
sd15 = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16)
sd15 = sd15.to('cuda')
sd15.enable_attention_slicing()
sd15.enable_xformers_memory_efficient_attention()

In [ ]:
## Remove Black and white Photos
num_images = 3

prompt = 'photograph of an old car'
neg_prompt = 'bw photo'

imgs = sd15(prompt, negative_prompt = neg_prompt, num_images_per_prompt=num_images).images

grid = grid_img(imgs, rows = 1, cols = 3, scale=0.75)
grid

In [ ]:
sd2 = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1", torch_dtype=torch.float16)
sd2 = sd2.to("cuda")
sd2.enable_attention_slicing()
sd2.enable_xformers_memory_efficient_attention()

In [ ]:
## Remove Black and white Photos
num_images = 3

prompt = 'photograph of an old car'
neg_prompt = 'bw photo'

imgs = sd2(prompt, negative_prompt = neg_prompt, num_images_per_prompt=num_images).images

grid = grid_img(imgs, rows = 1, cols = 3, scale=0.75)
grid

In [ ]:
seed = 777
prompt = "a photo of a terrier wearing sunglasses, on the beach, ocean in the background"
generator = torch.Generator('cuda').manual_seed(seed)
img = sd2(prompt, generator=generator).images[0]
img

In [ ]:
sd2.scheduler.compatibles

In [ ]:
sd2.scheduler.config

In [ ]:
from diffusers import LMSDiscreteScheduler
sd2.scheduler = LMSDiscreteScheduler.from_config(sd2.scheduler.config)
generator = torch.Generator(device = 'cuda').manual_seed(seed)
img = sd2(prompt, num_inference_steps = 60, generator=generator).images[0]
img

In [ ]:
from diffusers import EulerAncestralDiscreteScheduler

sd2.scheduler = EulerAncestralDiscreteScheduler.from_config(sd2.scheduler.config)

generator = torch.Generator(device="cuda").manual_seed(seed)
img = sd2(prompt, generator=generator, num_inference_steps=50).images[0]
img